In [31]:
import time
import os
import torch
import cv2
import sys
import json
import re
import yt_dlp
from google.colab import userdata
from google import genai
import numpy as np
from datetime import datetime
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video

In [32]:
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.enable_model_cpu_offload()

pipe.enable_vae_slicing()

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import yt_dlp

def download_youtube_video(link, output_path):
  ydl_opts = {
      'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
      'format': 'bestvideo/best',
  }

  try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download([link])
    print(f"Video downloaded successfully to: {output_path}")
  except yt_dlp.utils.DownloadError as e:
    print(f"Error downloading video: {e}")


# Example usage
video_link = 'https://www.youtube.com/watch?v=NuyQTmFSj9A'
download_location = '/home/downloads/'

#check if the path exists, if not create it.
if not os.path.exists(download_location):
    os.makedirs(download_location)

download_youtube_video(video_link, download_location)


In [45]:
client = genai.Client(api_key=userdata.get('GENAI_API_KEY'))

def process_video(fileLocation, userInput):
    # If an upload reference exists, it can be used; otherwise, upload the file.
    uploadVideo = None  # Set to a URL if already uploaded; else, None to upload the file.
    video_file = None

    if uploadVideo:
        print("Using previously uploaded file...")
        video_file = uploadVideo
    else:
        print("Uploading file...")
        video_file = client.files.upload(file=fileLocation)
        print(f"Completed upload: {video_file.uri}")

        # Wait until the file is processed.
        while video_file.state.name == "PROCESSING":
            print('.', end='')
            time.sleep(1)
            video_file = client.files.get(name=video_file.name)

        if video_file.state.name == "FAILED":
            raise ValueError(video_file.state.name)
        print('Done')

    inputContent = """
    - You are a movie director.
    - The video has already been shot and now you are tasked to change the script of a movie.
    - First and foremost, you need to understand the video completely.
    - Once you are done completely understanding the video, you will need to change the script according to the input i give you wrapped in triple colons(:).
    - take you time and think about my input and how i want to change the script.
    - IF THE OUTPUT IS GREAT YOU WILL BE REWARDED WITH A HUGE BONUS.

    jsonObject = { "summary": "<point 1 output>", "changes": "<point 2 output>", "perSceneDescription": "<point 3 output>", "scenes": "POINT 4 output which should be an array in place of this string" }

    STRICTLY FOLLOW THIS jsonObject as output format. DO NOT CHANGE THE KEY NAMES. DO NOT CHANGE THE OUTPUT FORMAT. IF YOU CHANGE THE FORMAT YOU WILL NOT BE REWARDED.

    PLEASE MAKE SURE THAT THE OUTPUT JSON IS IN THE CORRECT FORMAT.

    In output JSON, I want 4 things from you
    1. Summarize this video. USE KEYNAME summary in the json object.
    2. Give me the changes you are thinking to put in the script based on my input. USE KEY changes IN THE JSON OBJECT
    3. Give me scene by scene description of the video that can be used to generate a new video. Each scene should be atmax 2 seconds long. you can create multiple scenes.
    4. Give me the updated/modified script of the video. THIS OUTPUT SHOULD STICTLY BE WITH KEY AS 'scenes' WHICH IS AN ARRAY OF THE scenes! DO NOT CHANGE THE KEY NAME. DO NOT SEND THIS POINT'S OUTPUT OTHER THAN THIS FORMAT. ARRAY ELEMENT WILL BE TEXT ONLY OF THE SCENE. EACH SCENE SHOULD BE VERY DETAILED. EVERY SINGLE DETAIL TO BE ADDED. SCENE TEXT SHOULD BE ATLEAST 50 WORDS LONG BUT NOT MORE THAN 70 WORDS. EACH SCENE SHOULD NOT BE MORE THAN 70 WORDS!. EACH SCENE'S DESCRIPTION SHOULD BE CONNECTED TO ONE ANOTHER SO WE CAN TRANSITION DURING THE GENERATION OF THE VIDEO. THIS CAN BE SKIPPED IF AND ONLY IF THE CURRENT SCENE HAS NO RELATION TO ITS PREVIOUS SCENE."""

    userInputPlaceHolder = inputContent + ":::" + userInput + ":::"

    response = client.models.generate_content(
        model="gemini-1.5-pro",
        contents=[
            video_file,
            userInputPlaceHolder
        ]
    )
    print(response.text)
    # Markdown(response.text)
    return response.text

In [34]:
video_url = "/home/downloads/Days - Silent Short Film.webm"

In [35]:
prompt = "Change the story in such a way that in the end after seeing the last pill, the protagonist changes its lifestyles and starts engaging in things they enjoy doing eg playing guitar and scetching."

In [46]:
response_text = process_video(video_url, prompt)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/qjpxii4bjq55
....Done
Here's the updated script based on your request:

```json
{
  "summary": "A young man wakes up, checks his phone, performs his morning routine, gets dressed for work, and takes a pill from a jar labeled \"DAYS.\"  He repeats this process for a year, the jar gradually emptying. After the last pill, the story shifts, and he begins to pursue his hobbies of playing guitar and sketching.",
  "changes": "The original story implies a mundane, repetitive existence symbolized by the daily pill. The requested change introduces a positive shift at the end. Instead of continuing the routine, the protagonist breaks free after the final pill and embraces his passions.",
  "perSceneDescription": [
    {"start": 0, "end": 2, "description": "The young man wakes up in bed, checks his phone, and rubs his eyes."},
    {"start": 2, "end": 4, "description": "He gets out of bed and goes to the kit

In [47]:
pureJson = None

json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
if json_match:
    pureJson = json_match.group(0)
# oldData = json.loads(pureJson)
# print(oldData.get("scenes"))

In [48]:
scenes = None
try:
    data = json.loads(pureJson)
    scenes = data.get("scenes")
except json.JSONDecodeError as e:
    print("Failed to parse response as JSON:", e)
    print("Raw response:", response_text)

Failed to parse response as JSON: Expecting ',' delimiter: line 17 column 16 (char 1682)
Raw response: Here's the updated script based on your request:

```json
{
  "summary": "A young man wakes up, checks his phone, performs his morning routine, gets dressed for work, and takes a pill from a jar labeled \"DAYS.\"  He repeats this process for a year, the jar gradually emptying. After the last pill, the story shifts, and he begins to pursue his hobbies of playing guitar and sketching.",
  "changes": "The original story implies a mundane, repetitive existence symbolized by the daily pill. The requested change introduces a positive shift at the end. Instead of continuing the routine, the protagonist breaks free after the final pill and embraces his passions.",
  "perSceneDescription": [
    {"start": 0, "end": 2, "description": "The young man wakes up in bed, checks his phone, and rubs his eyes."},
    {"start": 2, "end": 4, "description": "He gets out of bed and goes to the kitchen."},
 

In [55]:
scenes = [
    "A young man, MARK (20s), stirs awake in his sparsely decorated bedroom. Sunlight streams through the blinds. He reaches for his phone on the nightstand, checks the screen, and rubs the sleep from his eyes with a sigh.",
    "Mark throws back the covers and gets out of bed.  He walks across the room to the kitchen, the floorboards creaking slightly under his bare feet. The room is dimly lit, with the only source of light from the small window above the sink.",
    "In the kitchen, Mark opens a cabinet revealing plain white mugs and bowls.  He takes out a mug and sets it on the counter. The cabinet doors are old and worn, with some of the paint chipping off.",
    "Mark pours coffee from a plain white carafe into his mug. Steam rises, briefly obscuring his face. He pauses, lost in thought, staring into the dark liquid.",
    "Mark splashes water onto his face in the bathroom sink.  The fluorescent light above casts harsh shadows.  He looks tired and worn down, with dark circles under his eyes.",
    "He squeezes toothpaste onto his toothbrush and begins brushing his teeth vigorously. He stares at his reflection in the mirror, his expression blank and unreadable.",
    "Mark stands before a mirror, dressed in a crisp dark suit and white shirt. He meticulously knots his tie, adjusting it with precision. His face is reflected in the mirror, revealing his serious expression.",
    "Mark walks to a kitchen counter where a clear glass jar sits. A handwritten label reads DAYS. He opens the jar and takes out a small, brightly colored pill which he swallows without water.",
    "The same bedroom, a year later. Mark wakes up and immediately reaches for his phone. He scrolls through notifications with a slightly more alert expression than before.",
    "Mark brushes his teeth, looking at his reflection in the mirror. There's a hint of a smile playing on his lips. A new tube of toothpaste can be seen near the edge of the sink.",
    "He puts on his suit, but this time he pauses and gives himself a quick, appraising look, adjusting his jacket with a touch more confidence. There's a new picture frame hanging on the wall, and some other decorations visible.",
    "Mark approaches the DAYS jar, now almost empty. He opens the jar, and this time, he hesitates, looking thoughtfully at the remaining pills. He closes the jar and places it on the kitchen counter, leaving it there untouched.",
    "Close-up of Mark's hand holding a single, brightly colored pill, the very last one. He contemplates it for a moment, turning it over in his fingers.",
    "With a decisive motion, Mark drops the last pill into the jar. The small sound it makes echoes in the now-quiet kitchen.",
    "Mark picks up his guitar from a stand in the corner of his living room.  Dust motes dance in the sunlight filtering through the window. He strums a few chords, a smile spreading across his face.",
    "Mark sits at a small table, sketching in a pad. His pencil moves fluidly across the page, capturing the light and shadow playing on a nearby vase of flowers.  He's engrossed in his work, humming softly to himself."
  ]

In [57]:
sceneIndex = 0
def concatenate_videos(video_directory, output_path):
    video_files = [f for f in os.listdir(video_directory) if f.endswith(('.mp4', '.avi', '.mov'))]  # Add more extensions if needed
    # video_files.sort() # Ensure videos are processed in the correct order

    # print("file order: ", video_files)

    if not video_files:
        print(f"No video files found in {video_directory}")
        return

    first_video = cv2.VideoCapture(os.path.join(video_directory, video_files[0]))
    fps = first_video.get(cv2.CAP_PROP_FPS)
    width = int(first_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(first_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Or other codecs like 'XVID'
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for index in range (0, len(video_files)):
        # video_path = os.path.join(video_directory, video_file)
        video_path = os.path.join(video_directory, f"{index}.mp4")
        index += 1
        video = cv2.VideoCapture(video_path)
        while video.isOpened():
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)
        video.release()

    first_video.release()
    out.release()
    print(f"Combined video saved to {output_path}")

custom_video_dir = '/home/videos/v1'
output_path = '/home/videos/v1/combined_video.mp4'

for scene in scenes:
    prompt = scene
    video_frames = pipe(prompt, num_inference_steps=10, num_frames=8).frames[0]

    os.makedirs(custom_video_dir, exist_ok=True)

    internal_output_path = os.path.join(custom_video_dir, f"{sceneIndex}.mp4")

    video_path = export_to_video(video_frames, internal_output_path)

    print(f"Video saved to: {video_path}")
    sceneIndex += 1


concatenate_videos(custom_video_dir, output_path)

  0%|          | 0/10 [00:00<?, ?it/s]

Video saved to: /home/videos/v1/0.mp4


  0%|          | 0/10 [00:00<?, ?it/s]

Video saved to: /home/videos/v1/1.mp4


  0%|          | 0/10 [00:00<?, ?it/s]

...

Video saved to: /home/videos/v1/13.mp4


  0%|          | 0/10 [00:00<?, ?it/s]

Video saved to: /home/videos/v1/14.mp4


  0%|          | 0/10 [00:00<?, ?it/s]

Video saved to: /home/videos/v1/15.mp4
Combined video saved to /home/videos/v1/combined_video.mp4
